In [1]:
# import the usual suspects
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = [12.0, 9.0]

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 1000)

In [2]:
#t_itm = pd.read_csv('../data/' + 't_itm.csv', low_memory=False)
bidding_info = pd.read_csv('../data/' + 'bidding_info.csv', low_memory=False)
change_orders = pd.read_csv('../data/' + 'change_orders.csv', low_memory=False)
#cont_itm = pd.read_csv('../data/' + 'cont_itm.csv', low_memory=False)
#project_dates = pd.read_csv('../data/' + 'project_dates.csv', low_memory=False)

In [3]:
bidding_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10969 entries, 0 to 10968
Data columns (total 9 columns):
Vendor Number                         8955 non-null object
Proposal Number                       10969 non-null object
Vendor Name                           8955 non-null object
Bid Total                             8299 non-null object
Engineers Estimate                    8955 non-null object
Difference                            10969 non-null object
Percentage Bid Total Over Estimate    10969 non-null object
Awarded                               8955 non-null float64
Letting Date                          8955 non-null object
dtypes: float64(1), object(8)
memory usage: 771.3+ KB


In [4]:
# drop Averages from bidding_info
bidding_info = bidding_info[bidding_info['Awarded'].notnull()]

In [5]:
bidding_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8955 entries, 0 to 10966
Data columns (total 9 columns):
Vendor Number                         8955 non-null object
Proposal Number                       8955 non-null object
Vendor Name                           8955 non-null object
Bid Total                             8299 non-null object
Engineers Estimate                    8955 non-null object
Difference                            8955 non-null object
Percentage Bid Total Over Estimate    8955 non-null object
Awarded                               8955 non-null float64
Letting Date                          8955 non-null object
dtypes: float64(1), object(8)
memory usage: 699.6+ KB


In [28]:
bidding_info.head()

,Vendor Number,Proposal Number,Vendor Name,Bid Total,Engineers Estimate,Difference,Percentage Bid Total Over Estimate,Awarded,Letting Date
0,159C,C13970R-ALT A,KIEWIT WESTERN CO.,11117898.98,9692453.81,1425445.17,14.71%,0.0,1/20/2005
2,074A,C14951,CRUZ CONSTRUCTION INCORPORATED,1697764.32,1006169.00,691595.32,68.74%,0.0,1/27/2005
3,447A,C14951,IDEAL FENCING CORPORATION,1243328.10,1006169.00,237159.10,23.57%,1.0,1/27/2005
4,159C,C14951,KIEWIT WESTERN CO.,1854136.00,1006169.00,847967.00,84.28%,0.0,1/27/2005
6,060A,MM3022,"HIGHWAY TECHNOLOGIES, INC.",783966.60,695544.00,88422.60,12.71%,0.0,1/27/2005


In [56]:
pd.to_datetime(bidding_info['Letting Date']).dt.month  pd.to_datetime(bidding_info['Letting Date']).dt.year

0        2006
2        2006
3        2006
4        2006
6        2006
7        2006
9        2007
10       2007
11       2007
12       2007
13       2007
14       2007
16       2007
17       2007
18       2007
19       2007
20       2007
21       2007
22       2007
23       2007
25       2007
26       2007
27       2007
28       2007
29       2007
30       2007
31       2007
33       2007
34       2007
35       2007
36       2007
38       2007
39       2007
40       2007
41       2007
43       2007
44       2007
45       2007
46       2007
47       2007
49       2007
50       2007
51       2007
53       2007
54       2007
55       2007
57       2007
58       2007
59       2007
60       2007
61       2007
62       2007
64       2007
65       2007
66       2007
67       2007
69       2007
70       2007
71       2007
72       2007
73       2007
75       2007
76       2007
77       2007
79       2007
80       2007
81       2007
82       2007
83       2007
84       2007
85       2007
87    

In [69]:
bidding_info['month'] = pd.to_datetime(bidding_info['Letting Date']).dt.month
bidding_info['year'] = pd.to_datetime(bidding_info['Letting Date']).dt.year

bidding_info['date']= bidding_info.apply(lambda x:pd.Timestamp.strptime("{0} {1}".format(x['year'],x['month']), "%Y %m"),axis=1)


In [79]:
bidding_info.columns
cols_to_drop = ['Vendor Number', 'Vendor Name', 'Difference',
       'Percentage Bid Total Over Estimate', 'Letting Date',
       'month', 'year']
bidding_info = bidding_info.drop(cols_to_drop, axis=1)
bidding_info.columns


Index(['Proposal Number', 'Bid Total', 'Engineers Estimate', 'Awarded',
       'date'],
      dtype='object')

In [6]:
# Converting all fo the $ amount columns into numbers in
# bidding_info  - in place

def dollar_val(strng, mylocale='en_US.UTF-8'):
    '''Function to convert formatted string in database to float of dollar amounts
    Generalized to any locale for possible reutilization in the future
    '''
    import locale
    locale.setlocale(locale.LC_ALL, mylocale)
    locale._override_localeconv = {'n_sign_posn':0}
    conv = locale.localeconv()
    if type(strng) == str:
        raw_numbers = strng.replace(conv['currency_symbol'], '')
    else:
        raw_numbers = strng
    return locale.atof(raw_numbers)


columns_to_convert = ['Bid Total', 'Engineers Estimate', 'Difference']
for col in columns_to_convert:
    tmp = bidding_info[col]
    bidding_info[col] = tmp.map(dollar_val, na_action='ignore')

In [25]:
columns_to_convert = ['C_O_AMT']
for col in columns_to_convert:
    tmp = change_orders[col]
    change_orders[col] = tmp.map(dollar_val, na_action='ignore')

In [26]:
change_orders['date'] = change_orders.apply(lambda 
                        x:pd.Timestamp.strptime("{0}".format(x['APPR_DT']//100),
                         "%Y%m"),axis=1)

In [84]:
bidding_info[bidding_info['Proposal Number'] == 'C11873']

,Proposal Number,Bid Total,Engineers Estimate,Awarded,date
7202,C11873,17756068.60,17101039.0,0.0,2011-10-01
7203,C11873,17641300.00,17101039.0,0.0,2011-10-01
7204,C11873,17034895.69,17101039.0,1.0,2011-10-01
7205,C11873,17592200.52,17101039.0,0.0,2011-10-01
7206,C11873,19346156.95,17101039.0,0.0,2011-10-01
7207,C11873,19537303.00,17101039.0,0.0,2011-10-01
7208,C11873,17303850.74,17101039.0,0.0,2011-10-01


In [27]:
change_orders.head()

,Vendor,Bid Amount,CONT_ID,LEV2_OFFICE_NBR,DESC1,C_O_NBR,CD_DESC,APPR_DT,C_O_AMT,CHNG_DESC,C_O_T,date
0,"CONCRETE EXPRESS, INC.","$17,034,895.69",C11873,4,CHERRYVALE RD. TO N. 75TH ST.,7,Modifications With No CMO/MCR Required,20120606,4300.00,Added field lab,00,2012-06-01
1,"CONCRETE EXPRESS, INC.","$17,034,895.69",C11873,4,CHERRYVALE RD. TO N. 75TH ST.,8,Modifications With No CMO/MCR Required,20120607,3000.00,FA for Century Link,00,2012-06-01
2,"CONCRETE EXPRESS, INC.","$17,034,895.69",C11873,4,CHERRYVALE RD. TO N. 75TH ST.,9,Modifications With No CMO/MCR Required,20120607,500.00,FA for Tetra Tech,00,2012-06-01
3,"CONCRETE EXPRESS, INC.","$17,034,895.69",C11873,4,CHERRYVALE RD. TO N. 75TH ST.,10,Modifications With No CMO/MCR Required,20120606,10079.59,FA Trench Box Invoice,00,2012-06-01
4,"CONCRETE EXPRESS, INC.","$17,034,895.69",C11873,4,CHERRYVALE RD. TO N. 75TH ST.,11,Modifications With No CMO/MCR Required,20120607,3000.00,F/A Xcel Gas,00,2012-06-01


In [80]:
change_orders.columns

Index(['Vendor', 'Bid Amount', 'CONT_ID', 'LEV2_OFFICE_NBR', 'DESC1',
       'C_O_NBR', 'CD_DESC', 'APPR_DT', 'C_O_AMT', 'CHNG_DESC', 'C_O_T',
       'date'],
      dtype='object')

In [7]:
# Split dataframe into winers and losers

winners = bidding_info[bidding_info['Awarded'] == 1].drop(['Awarded'], axis=1)
losers = bidding_info[bidding_info['Awarded'] == 0].drop(['Awarded'], axis=1)

#create groupby onjects on losers and on original dataframe (minus averages)

g_losers = losers.groupby('Proposal Number')
g_bidding_info = bidding_info.groupby('Proposal Number')

In [8]:
losers = pd.DataFrame(g_losers.agg(['size', 'mean', 'min', 'max']))

In [ ]:
loser = pd.DataFrame(bidding_info[bidding_info['Awarded'] == 1].drop(['Awarded'], axis=1).groupby('Proposal Number')

In [9]:
losers.columns = [' '.join(col) for col in losers.columns]

In [10]:
losers.reset_index(inplace=True)

In [11]:
print(losers.size)
print(losers['Proposal Number'].unique().size)

24869
1913


In [12]:
print(winners.size)
print(winners['Proposal Number'].unique().size)

14784
1848


In [13]:
print(bidding_info.size)
print(bidding_info['Proposal Number'].unique().size)

80595
2013


In [14]:
won_proposals = set(winners['Proposal Number'].unique())
won_proposals

{'C18236',
 'C17303',
 'C17046',
 'C18763R',
 'C14890',
 'C19280',
 'C19798',
 'C21163',
 'C17753',
 'C17820',
 'C17234',
 'C21414',
 'MM2032',
 'C17836',
 'C16148',
 'MM3022',
 'C19314',
 'C20099',
 'C16414',
 'C20072',
 'C19356R',
 'C20804',
 'C15914',
 'C15585',
 'C15591',
 'C12810',
 'C19919',
 'C17605',
 'C17938',
 'C15555',
 'C19549',
 'C16058',
 'C17060',
 'C13698',
 'C15916',
 'C17713',
 'C18900',
 'C18082',
 'C15566',
 'C15209R',
 'C15922B',
 'C17452',
 'C20463',
 'C18786',
 'C18013',
 'C14087',
 'MM5049',
 'C17658',
 'C17987',
 'C19098',
 'C20218',
 'C17202',
 'C19910',
 'C13697',
 'C18269',
 'C15809',
 'C19898',
 'C17358',
 'C15107R',
 'C18275',
 'C15023',
 'C17770',
 'C18029',
 'MM5050',
 'C16190',
 'C19326',
 'C19312',
 'C16522',
 'C18866',
 'C19217',
 'C17208',
 'C17772',
 'C17242',
 'C18537',
 'C19652',
 'C18295-ALT',
 'C16052',
 'C19708',
 'C17377',
 'C17216',
 'C18596',
 'C16103',
 'C20168',
 'C19706',
 'C19160',
 'C19067',
 'C19395',
 'C17535',
 'C15771',
 'C18283',
 

In [19]:
change_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6633 entries, 0 to 6632
Data columns (total 11 columns):
Vendor             6633 non-null object
Bid Amount         6633 non-null object
CONT_ID            6633 non-null object
LEV2_OFFICE_NBR    6633 non-null int64
DESC1              6633 non-null object
C_O_NBR            6633 non-null int64
CD_DESC            6633 non-null object
APPR_DT            6633 non-null int64
C_O_AMT            6633 non-null object
CHNG_DESC          6633 non-null object
C_O_T              6633 non-null object
dtypes: int64(3), object(8)
memory usage: 570.1+ KB


In [73]:
change_orders['date'].max()

Timestamp('2016-12-01 00:00:00')

In [74]:
change_orders['date'].min()

Timestamp('2010-01-01 00:00:00')

In [75]:
bidding_info['date'].min()

Timestamp('2005-01-01 00:00:00')

In [76]:
bidding_info['date'].max()

Timestamp('2016-12-01 00:00:00')